In [2]:
# Imports

# import itertools
from itertools import permutations
import math
# import os
import pickle
import platform
import random
from tkinter import Tk

# from cvxopt import solvers, matrix
from matplotlib import animation
from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import torch
# import torchvision
# from torchvision import transforms, models,datasets
# from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# from tools import *
import tools

# Extend width of Jupyter Notebook Cell to the size of browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# OS related settings
if platform.system() == 'Windows':
    print('Windows')
#     %matplotlib tk
    %matplotlib qt
elif platform.system() == 'Darwin':
    print('macOS')
    Tk().withdraw()
    %matplotlib osx
elif platform == 'linux' or platform == 'linux2':
    print('Linux')
# This line of "print" must exist right after %matplotlib command, otherwise JN will hang on the first import statement after this.
print('Interactive plot activated')

macOS
Interactive plot activated


In [3]:
# Parameters
num_repetition = 4

dim_list = list(range(4, 5))

# num_per_perm_list_train = [30]
# num_per_perm_list_train = [1, 10]
num_per_perm_list_train = [1, 10, 50, 100] # Every permutation gets the same number of train/test data samples
num_per_perm_list_test = [10] # Every permutation gets the same number of train/test data samples

superset_factor = 4 # Create a superset # times larger than what is needed to ensure each class would have sufficient number of samples

distributions = ['uniform', 'normal', 'bimodal']
distribution = distributions[0]

weights = {3: np.asarray([[0.8, 0.1, 0.1],
                          [0.2, 0.7, 0.1],
                          [0.5, 0.2, 0.3],
                          [0.1, 0.3, 0.5]]),
           4: np.asarray([[0.1, 0.2, 0.3, 0.4],
                          [0.2, 0.2, 0.2, 0.4],
                          [0, 0.1, 0.3, 0.6],
                          [0, 0, 0.4, 0.6]]),
           5: np.asarray([[0.1, 0.1, 0.1, 0.2, 0.5],
                          [0, 0, 0.4, 0.3, 0.3],
                          [0, 0, 0, 0.4, 0.6],
                          [0.1, 0.8, 0.1, 0, 0]]),
           6: np.asarray([[0.1, 0.1, 0.1, 0.1, 0.2, 0.4],
                          [0, 0, 0.3, 0.3, 0.3, 0.1],
                          [0, 0, 0, 0, 0.4, 0.6],
                          [0.1, 0.7, 0.1, 0.1, 0, 0]]),}

avg_funcs = [np.amin, np.amax, np.mean, None, None, None, None] # List of avg functions

avg_names = {}
for dim in dim_list:
    weight = weights[dim]
    weight_legend = []
    for w in weight:
        weight_legend.append(' '.join(map(str, (w*10).astype(int))))
    avg_names[dim] = ['Min', 'Max', 'Mean'] + weight_legend

train_group_num_limit = math.factorial(4)

models = [tools.Choquet_Integral_QP]
model_names = ['QP']

# models = [tools.Choquet_Integral_NN]
# model_names = ['NN']

# models = [tools.Choquet_Integral_QP, tools.Choquet_Integral_NN]
# model_names = ['QP', 'NN']

# models = [tools.Choquet_Integral_QP, tools.Choquet_Integral_QP]
# model_names = ['QP', 'QP']


output_dir = 'output/'

In [5]:
# Run
import timeit
start0 = timeit.default_timer()
num_tqdm = num_repetition*len(dim_list)*len(num_per_perm_list_train)
print(num_tqdm)

# [Dim, Rep, #perPerm, % of perm, avg_func, model]
MSEs_seen_by_dim = {}
MSEs_unseen_by_dim = {}
FM_by_dim = {}



for rep in range(num_repetition):
    start1 = timeit.default_timer()
    train_data_source = tools.Data_Source(dim_list[-1], num_per_perm_list_train[-1], superset_factor, distribution)
    stop1 = timeit.default_timer()
    print('Time1: ', stop1 - start1)   
    train_superset = train_data_source.get_superset()
    
    start2 = timeit.default_timer()
    test_data_source = tools.Data_Source(dim_list[-1], num_per_perm_list_test[-1], superset_factor, distribution)
    stop2 = timeit.default_timer()
    print('Time2: ', stop2 - start2)   
    test_superset = test_data_source.get_superset()
    
    random_seed = random.random()
    
    for dim in dim_list:

        for avg_idx in range(len(weights[dim])):
            avg_funcs[3+avg_idx] = tools.w_avg(weights[dim][avg_idx])

        all_perms = list(permutations(list(range(dim)))) # N! possible permutations
#         random.Random(rep*random_seed).shuffle(all_perms)
        random.shuffle(all_perms)
        print(all_perms)
        
        # When the # of possible permutations exceed certain number (in here 5!), 
        # instead of feeding only one more permutation a time, feed more.
        if len(all_perms) > train_group_num_limit:
            step = int(len(all_perms) / train_group_num_limit)
        else:
            step = 1

        # Mean Squared Error [for each repetition, for each avg function, for each model, for each percentage, for each data#perperm, for each dim], of all test samples, for both seen and unseen data.
        MSEs_seen = np.zeros((len(num_per_perm_list_train), len(range(step-1, len(all_perms), step)), len(avg_funcs), len(models)))
        MSEs_unseen = np.zeros((len(num_per_perm_list_train), len(range(step-1, len(all_perms), step))-1, len(avg_funcs), len(models)))
        # Record FM after train session with both seen and unseen data pattern
        FM = np.zeros((len(num_per_perm_list_train), len(range(step-1, len(all_perms), step)), len(avg_funcs), len(models), 2**dim-1))

    
        for npp_idx, num_per_perm in enumerate(num_per_perm_list_train):
            
            train_idx_by_perm = train_data_source.get_data_idx(dim, num_per_perm)
            test_idx_by_perm = test_data_source.get_data_idx(dim, num_per_perm_list_test[0])
            start3 = timeit.default_timer()
            for perc_idx, perc in enumerate(tqdm(range(step-1, len(all_perms), step))):
                
#             for perc_idx, perc in enumerate(range(step-1, len(all_perms), step)):
                # Find index of train/test sample in superset and shuffle
                train_idx = np.concatenate(train_idx_by_perm[0:perc+1])
                np.random.shuffle(train_idx)
                test_idx = np.concatenate(test_idx_by_perm[0:perc+1])
                # Find data sample through index
                train_d = train_superset[:, train_idx][0:dim]
                test_d = test_superset[:, test_idx][0:dim]
                # Define unseen test data samples when the train data doesn't cover 100% of the permutation
                if perc < len(all_perms)-1:
                    test_idx_unseen = np.concatenate(test_idx_by_perm[perc+1:])
                    test_d_unseen = test_superset[:, test_idx_unseen][0:dim]
                else:
                    test_d_unseen = []

                # Define subsets of 'X', or keys for fuzzy measure. Like '1 2' or '1 3 4 5' for g(x1, x2) or g(x1, x3, x4, x5)
                sourcesInNode, subset = tools.sources_and_subsets_nodes(dim)
                keys = [str(np.sort(i)+1) for i in sourcesInNode]
                
                for avg_idx, avg_func in enumerate(avg_funcs):
                    start4 = timeit.default_timer()
                    # Calculate label with given avg function
                    train_label = avg_func(train_d, 0)
                    test_label = avg_func(test_d, 0)
                    
                    for model_idx, model in enumerate(models):
                        start5 = timeit.default_timer()
                        if model_names[model_idx] == 'QP':
                            # Initialize ChIQP
                            chi_model = model()
                            # Train 
                            chi_model.train_chi(train_d, train_label)
                            # Get fuzzy measure learned
                            fm = chi_model.fm
                            
                        elif model_names[model_idx] == 'NN':
                            # Initialize ChINN
                            chi_model = model(dim, 1)
                            # Parameters for training NN
                            lr = 0.05 # Learning rate
                            num_epoch = 100
                            criterion = torch.nn.MSELoss(reduction='mean')
                            optimizer = torch.optim.SGD(chi_model.parameters(), lr=lr)
                            
                            # Train 
                            tools.train_chinn(chi_model, lr, criterion, optimizer, num_epoch, torch.tensor(train_d, dtype=torch.float), torch.tensor(train_label, dtype=torch.float))
                            # Get fuzzy measure learned
                            FM_learned = (chi_model.chi_nn_vars(chi_model.vars).cpu()).detach().numpy()
                            fm_dict_binary = dict(zip(keys, FM_learned[:,0]))
                            fm_dict_lexicographic = tools.get_keys_index(dim)
                            for key in fm_dict_lexicographic.keys():
                                fm_dict_lexicographic[key] = fm_dict_binary[key]
                            fm = fm_dict_lexicographic
                            
                        stop5 = timeit.default_timer()
#                         print('Time5: ', stop5 - start5)
                        
                        FM[npp_idx, perc_idx, avg_idx, model_idx, :] = np.asarray(list(fm.values()))
                        # Calculate result from integral with test data
                        test_output = np.apply_along_axis(tools.get_cal_chi(fm), 0, test_d)
                        MSE = ((test_output - test_label)**2).mean()
                        MSEs_seen[npp_idx, perc_idx, avg_idx, model_idx] = MSE
                        # Calculate result from integral with test data - unseen
                        if perc < len(all_perms)-1:
                            test_label_unseen = avg_func(test_d_unseen, 0)
                            test_out_unseen = np.apply_along_axis(tools.get_cal_chi(fm), 0, test_d_unseen)
                            MSEs_unseen[npp_idx, perc_idx, avg_idx, model_idx] = ((test_out_unseen - test_label_unseen)**2).mean()
                    stop4 = timeit.default_timer()
#                     print('Time4: ', stop4 - start4)
#                     print('Time5/4: ', (stop5-start5)/(stop4-start4))
            stop3 = timeit.default_timer()
            print('Time3: ', stop3 - start3)  
        if dim in FM_by_dim.keys():
            FM_by_dim[dim] = np.append(FM_by_dim[dim], np.expand_dims(FM, axis=0), axis=0)
            MSEs_seen_by_dim[dim] = np.append(MSEs_seen_by_dim[dim], np.expand_dims(MSEs_seen, axis=0), axis=0)
            MSEs_unseen_by_dim[dim] = np.append(MSEs_unseen_by_dim[dim], np.expand_dims(MSEs_unseen, axis=0), axis=0)
        else:
            FM_by_dim[dim] = np.expand_dims(FM, axis=0)
            MSEs_seen_by_dim[dim] = np.expand_dims(MSEs_seen, axis=0)
            MSEs_unseen_by_dim[dim] = np.expand_dims(MSEs_unseen, axis=0)

    
stop0 = timeit.default_timer()
print('Time0: ', stop0 - start0)


16
Time1:  0.027438848999736365
Time2:  0.0052636799996435
[(0, 2, 3, 1), (0, 3, 2, 1), (3, 0, 1, 2), (2, 3, 1, 0), (3, 1, 2, 0), (2, 0, 3, 1), (1, 0, 3, 2), (3, 0, 2, 1), (0, 2, 1, 3), (1, 2, 3, 0), (3, 2, 0, 1), (2, 1, 3, 0), (3, 1, 0, 2), (2, 1, 0, 3), (2, 0, 1, 3), (0, 1, 2, 3), (1, 2, 0, 3), (2, 3, 0, 1), (3, 2, 1, 0), (0, 3, 1, 2), (1, 3, 2, 0), (0, 1, 3, 2), (1, 3, 0, 2), (1, 0, 2, 3)]


100%|██████████| 24/24 [00:11<00:00,  2.10it/s]


Time3:  11.404983904000346


100%|██████████| 24/24 [00:14<00:00,  1.63it/s]


Time3:  14.709979129999738


...

Time3:  16.032608617999358


100%|██████████| 24/24 [00:30<00:00,  1.29s/it]


Time3:  30.98674755799948


100%|██████████| 24/24 [00:48<00:00,  2.01s/it]

Time3:  48.13338347700028
Time0:  453.3122175319995


In [ ]:
# Save
now = datetime.datetime.now().strftime("-%m-%d-%Y@%H.%M.%S")
with open(output_dir + 'ChI_saved_file' + now, 'wb') as f:
    pickle.dump(FM_by_dim, f)
    pickle.dump(MSEs_seen_by_dim, f)
    pickle.dump(MSEs_unseen_by_dim, f)

In [129]:
# Plot
plt.close('all')
# Plot for seen
for dim in dim_list:
    MSEs_seen = MSEs_seen_by_dim[dim]
    MSE_mean = np.mean(MSEs_seen, 0)
    MSE_max = np.max(MSEs_seen, 0)
    MSE_min = np.min(MSEs_seen, 0)
    
    num_perm = math.factorial(dim)
    
    for npp_idx in range(len(num_per_perm_list_train)):
        for model_idx in range(len(model_names)):
            fig, ax = plt.subplots()
            x = (np.arange(MSE_mean.shape[1])+1) / MSE_mean.shape[1]
            plt.plot(x, MSE_mean[npp_idx, :, :, model_idx])
            ax.set_title('MSE (seen), Model=' + model_names[model_idx] + ', Dim=' + str(dim) + ', NPP=' + str(num_per_perm_list_train[npp_idx]))
            
            ax.legend(avg_names[dim])
            
            ax.set_xlabel('Percentage of Seen Data')
            ax.set_ylabel('MSEs avg')
            ax.xaxis.set_major_formatter(FuncFormatter('{0:.0%}'.format))

            for avg_idx in range(len(avg_funcs)):
                plt.fill_between(x, MSE_min[npp_idx, :, avg_idx, model_idx], MSE_max[npp_idx, :, avg_idx, model_idx], alpha=0.1)
                
            plt.savefig(output_dir + model_names[model_idx] + ' Dim=' + str(dim) + ' NPP=' + str(num_per_perm_list_train[npp_idx]) + ' (MSE seen).png')
                
                
# Plot for unseen
for dim in dim_list:
    MSEs_unseen = MSEs_unseen_by_dim[dim]
    MSE_mean = np.mean(MSEs_unseen, 0)
    MSE_max = np.max(MSEs_unseen, 0)
    MSE_min = np.min(MSEs_unseen, 0)
    
    num_perm = math.factorial(dim)

    for npp_idx in range(MSE_mean.shape[0]):
        for model_idx in range(MSE_mean.shape[-1]):
            fig, ax = plt.subplots()
            x = (np.arange(MSE_mean.shape[1])+1) / (MSE_mean.shape[1]+1)
            plt.plot(x, MSE_mean[npp_idx, :, :, model_idx])
            ax.set_title('MSE (unseen), Model=' + model_names[model_idx] + ', Dim=' + str(dim) + ', NPP=' + str(num_per_perm_list_train[npp_idx]))
            
            ax.legend(avg_names[dim])
            
            ax.set_xlabel('Percentage of Seen Data')
            ax.set_ylabel('MSEs avg')
            ax.xaxis.set_major_formatter(FuncFormatter('{0:.0%}'.format))

            for avg_idx in range(len(avg_funcs)):
                plt.fill_between(x, MSE_min[npp_idx, :, avg_idx, model_idx], MSE_max[npp_idx, :, avg_idx, model_idx], alpha=0.1)
                
            plt.savefig(output_dir + model_names[model_idx] + ' Dim=' + str(dim) + ' NPP=' + str(num_per_perm_list_train[npp_idx]) + ' (MSE unseen).png')

In [130]:
# Video for FM change



for dim in dim_list:
    FM = FM_by_dim[dim]
    FM_mean = np.mean(FM, 0)
    FM_min = np.amin(FM, 0)
    FM_max = np.amax(FM, 0)
    
    avg_name = avg_names[dim]
    fm_targets = [tools.get_min_fm_target, tools.get_max_fm_target, tools.get_mean_fm_target, tools.get_w_avg_target(weights[dim][0]), tools.get_w_avg_target(weights[dim][1]), tools.get_w_avg_target(weights[dim][2]), tools.get_w_avg_target(weights[dim][3])]
    
    
    for npp_idx in range(len(num_per_perm_list_train)):
        for model_idx in range(len(model_names)):
            for avg_idx, fm_target in enumerate(fm_targets):
                fm_mean = FM_mean[npp_idx, :, avg_idx, model_idx, :]
                fm_min = FM_min[npp_idx, :, avg_idx, model_idx, :]
                fm_max = FM_max[npp_idx, :, avg_idx, model_idx, :]

                # First set up the figure, the axis, and the plot element we want to animate
                fig = plt.figure()
                ax = plt.axes(xlim=(0, np.size(fm_mean, -1)-1), ylim=(-0.1, 1.1))
                line, = ax.plot([], [], lw=2)
                ax.set_xlabel('Fuzzy Measure Key')
                ax.set_ylabel('Fuzzy Measure Value')
                ax.set_title(model_names[model_idx] + ', Dim=' + str(dim) + ', NPP=' + str(num_per_perm_list_train[npp_idx]) + ' ' + avg_name[avg_idx])

                # initialization function: plot the background of each frame
                def init():
                    x = list(range(np.size(fm_mean, -1)))
                    y = list(fm_target(dim).values())
                    plt.plot(x, y)
                    ax.legend(['FM Target'], loc=4)


                # animation function.  This is called sequentially
                def animate(i):
                    x = np.asarray(list(range(np.size(fm_mean, -1))))
                    y = fm_mean[i, :]
                    line.set_data(x, y)
                    ax.legend(['FM Predict (Seen data percentage ' + str("{0:.0%}".format((i+1)/np.size(fm_mean, 0))) + ')', 'FM Target'])
                    ax.collections = []
                    plt.fill_between(x, fm_min[i, :], fm_max[i, :], color='blue', alpha=0.1)
                    return line,

                # call the animator.  blit=True means only re-draw the parts that have changed.
                anim = animation.FuncAnimation(fig, animate, frames=np.size(fm_mean, 0), init_func=init(), interval=200, blit=True)

                # save the animation as an mp4.  This requires ffmpeg or mencoder to be
                # installed.  The extra_args ensure that the x264 codec is used, so that
                # the video can be embedded in html5.  You may need to adjust this for
                # your system: for more information, see
                # http://matplotlib.sourceforge.net/api/animation_api.html
                anim.save(output_dir + model_names[model_idx] + ', Dim=' + str(dim) + ', NPP=' + str(num_per_perm_list_train[npp_idx]) + ' w_AVG=' + avg_name[avg_idx] + '.mp4', fps=3)
        #         plt.show()
                plt.close('all')

In [85]:
plt.close('all')